### Project 1

#### Part A

Jerônimo de Abreu Afrange

- Select 30 stocks in the S&P 500 universe
- Collect daily returns for the past 10 years (2014-2023)
- Create both value-weighted and equally weighted portfolio
- Rebalance these portfolios in the first day of the month based on the information on the last day (1-day lag)
- Which portfolio has the highest turnover?
- Compute daily returns for both portfolios
- Compute the following statistics for these portfolios:
    - Annualized average return
    - Annualized standar deviation
    - Sharpe ratio
    - Information ratio (vs S&P500)
- Plot cumulative returns for both portfolios, S&P500 and risk-free




In [39]:
import pandas as pd
import dateutil.relativedelta

In [40]:
# AUXILIARY FUNCTIONS

# calculates the month number
def month_number(date, start_date):
    delta = dateutil.relativedelta.relativedelta(date, start_date)
    return delta.years * 12 + delta.months

In [41]:
# DATA LOADING AND INITIAL MANIPULATION

# creates the DataFrames for de S&P500 index and de stock prices
stocks_data = pd.read_csv("../data/stock_prices.csv")
index_data = pd.read_csv("../data/sp500_index.csv")

# removes the stocks of which we don't have data from 2014 to the current day
incomplete_stocks = stocks_data["Ticker"].value_counts()
incomplete_stocks = incomplete_stocks[incomplete_stocks < incomplete_stocks.max()]
stocks_data = stocks_data[~stocks_data["Ticker"].isin(incomplete_stocks.index)]

In [42]:
# CALCULATION OF THE DAILY RETURNS AND CREATION OF THE MONTH NUMBER COLUMN

# creation of the daily return column
stocks_data["RETURN"] = stocks_data["PX_LAST"] / stocks_data["PX_OPEN"] - 1

# creation of the month number column
stocks_data["Date"] = pd.to_datetime(stocks_data["Date"])
stocks_data["MONTH_NUMBER"] = stocks_data["Date"].apply(lambda x: month_number(x, stocks_data["Date"].min()))

# defines a few constants
INITIAL_DATE = stocks_data["Date"].min()
INITIAL_PORT_VALUE = 1e6

In [43]:
# CREATION OF THE VALUE WEIGHTED PORTFOLIO

# creates the porfolio DataFrame, which is a copy of the current stocks_data DataFrame
portfolio_vw = stocks_data.copy()

# creation of the column for the total market cap of the stocks and column of the share of the total
portfolio_vw["TOTAL_MKT_CAP"] = portfolio_vw.groupby("Date")["CUR_MKT_CAP"].transform("sum")
portfolio_vw["MKT_CAP_SHARE"] = portfolio_vw["CUR_MKT_CAP"] / portfolio_vw["TOTAL_MKT_CAP"]

# creation of the initial porfolio stock ammount for each stock, value for each stock position and total portfolio value
initial_date = portfolio_vw["Date"] == INITIAL_DATE
portfolio_vw["PORT_STOCK_AMT"] = None
portfolio_vw["PORT_STOCK_MKT_VAL_LAST"] = None
portfolio_vw.loc[initial_date, "PORT_STOCK_AMT"] = (INITIAL_PORT_VALUE * portfolio_vw["MKT_CAP_SHARE"]) / portfolio_vw["PX_OPEN"]
portfolio_vw.loc[initial_date, "PORT_STOCK_MKT_VAL_LAST"] = portfolio_vw["PORT_STOCK_AMT"] * portfolio_vw["PX_LAST"]
portfolio_vw["PORT_TOTAL_MKT_VAL_LAST"] = portfolio_vw.groupby("Date")["PORT_STOCK_MKT_VAL_LAST"].transform("sum")

# propagate the stock positions to the whole month period
initial_values = portfolio_vw[initial_date][["Ticker", "PORT_STOCK_AMT"]]
portfolio_vw = portfolio_vw.drop(columns="PORT_STOCK_AMT")
merged_values = portfolio_vw[portfolio_vw["MONTH_NUMBER"] == 0]
portfolio_vw = portfolio_vw[portfolio_vw["MONTH_NUMBER"] != 0]
merged_values = merged_values.merge(initial_values, on="Ticker")
portfolio_vw = pd.concat([merged_values, portfolio_vw])



In [44]:
portfolio_vw

,Date,PX_LAST,PX_OPEN,PX_HIGH,PX_LOW,PX_VOLUME,CUR_MKT_CAP,Ticker,RETURN,MONTH_NUMBER,TOTAL_MKT_CAP,MKT_CAP_SHARE,PORT_STOCK_MKT_VAL_LAST,PORT_TOTAL_MKT_VAL_LAST,PORT_STOCK_AMT
0,2014-01-02,76.27,76.04,76.500,75.900,5212460.0,134027.7199,DIS,0.003025,0,1.005353e+06,0.133314,133717.275159,996889.363117,1753.209324
1,2014-01-03,76.11,76.30,76.697,76.010,4061042.0,133746.5552,DIS,-0.002490,0,1.002988e+06,0.133348,None,0,1753.209324
2,2014-01-06,75.82,76.53,76.840,75.490,6816693.0,133236.9441,DIS,-0.009277,0,1.006224e+06,0.132413,None,0,1753.209324
3,2014-01-07,76.34,76.12,76.700,75.900,4511157.0,134150.7295,DIS,0.002890,0,1.018683e+06,0.131690,None,0,1753.209324
4,2014-01-08,75.22,76.13,76.280,74.810,10914858.0,132182.5763,DIS,-0.011953,0,1.018454e+06,0.129787,None,0,1753.209324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75167,2024-08-06,54.08,52.93,55.110,52.659,16405667.0,74061.2621,CMG,0.021727,127,3.359677e+06,0.022044,None,0,NaN
75168,2024-08-07,53.63,54.50,55.300,53.410,11238844.0,73444.9979,CMG,-0.015963,127,3.338005e+06,0.022003,None,0,NaN
75169,2024-08-08,54.32,54.00,54.890,53.740,9305774.0,74389.9363,CMG,0.005926,127,3.407948e+06,0.021828,None,0,NaN
75170,2024-08-09,55.55,54.90,56.140,54.530,11569747.0,76074.3918,CMG,0.011840,127,3.427590e+06,0.022195,None,0,NaN
